In [6]:
import json
import monai
import os
import numpy as np
from monai.data.box_utils import convert_box_mode

### boxes in world coordinate

In [7]:
# input_dataset_json = 'data_sample.json'
# output_dir = 'out_world_coord'
# input_box_mode = 'cccwhd'
# image_coordinate = False

### boxes in voxel coordinate

In [8]:
image_coordinate = True
image_data_root = 'original_sample'
input_box_mode = 'xyzxyz'
input_dataset_json = 'data_sample_xyzxyz_image-coordinate.json'
output_dir = 'out_voxel_coord'

In [9]:
def save_obj(vertices, faces, filename):
    with open(filename, "w") as f:
        for v in vertices:
            f.write("v {} {} {}\n".format(*np.array(v)))

        for t in faces:
            f.write("f {} {} {} {}\n".format(*(np.array(t) + 1)))

    return

In [10]:
with open(input_dataset_json) as f:
    input_dataset = json.load(f)

if image_coordinate:
    image_loader = monai.transforms.LoadImage(reader=None, image_only=False)

for key in input_dataset.keys():
        section = input_dataset[key]

        for _k in range(len(section)):
            box_data = section[_k]["box"]
            box_filename = section[_k]["image"]
            box_filename = box_filename.split(os.sep)[-1]
            print("-- {0:d}th case name:".format(_k + 1), box_filename)

            if image_coordinate:
                image_name = os.path.join(image_data_root, section[_k]["image"])
                image_data = image_loader(image_name)
                affine = image_data[1]["original_affine"]

                # convert to RAS coordinate system (required by 3D Slicer)
                for _i in range(3):
                    if affine[_i, _i] < 0:
                        affine[_i, _i] *= -1.0
                        affine[_i, 3] *= -1.0

            vertices = []
            faces = []
            _i = 0
            for _vec in box_data:
                vec = convert_box_mode(
                    np.expand_dims(np.array(_vec), axis=0),
                    src_mode=input_box_mode,
                    dst_mode="xyzxyz",
                )
                vec = vec.squeeze()
                xmin, ymin, zmin = vec[0], vec[1], vec[2]
                xmax, ymax, zmax = vec[3], vec[4], vec[5]

                if image_coordinate:
                    _out = affine @ np.transpose(np.array([xmin, ymin, zmin, 1]))
                    xmin, ymin, zmin = _out[0], _out[1], _out[2]

                    _out = affine @ np.transpose(np.array([xmax, ymax, zmax, 1]))
                    xmax, ymax, zmax = _out[0], _out[1], _out[2]

                vertices += [
                    (xmax, ymax, zmin),
                    (xmax, ymin, zmin),
                    (xmin, ymin, zmin),
                    (xmin, ymax, zmin),
                    (xmax, ymax, zmax),
                    (xmax, ymin, zmax),
                    (xmin, ymin, zmax),
                    (xmin, ymax, zmax),
                ]

                faces += [
                    (0 + 8 * _i, 1 + 8 * _i, 2 + 8 * _i, 3 + 8 * _i),
                    (4 + 8 * _i, 7 + 8 * _i, 6 + 8 * _i, 5 + 8 * _i),
                    (0 + 8 * _i, 4 + 8 * _i, 5 + 8 * _i, 1 + 8 * _i),
                    (1 + 8 * _i, 5 + 8 * _i, 6 + 8 * _i, 2 + 8 * _i),
                    (2 + 8 * _i, 6 + 8 * _i, 7 + 8 * _i, 3 + 8 * _i),
                    (4 + 8 * _i, 0 + 8 * _i, 3 + 8 * _i, 7 + 8 * _i),
                ]

                _i += 1

            save_obj(vertices, faces, os.path.join(output_dir, box_filename + ".obj"))

-- 1th case name: 1.3.6.1.4.1.14519.5.2.1.6279.6001.108197895896446896160048741492.mhd
-- 2th case name: 1.3.6.1.4.1.14519.5.2.1.6279.6001.109002525524522225658609808059.mhd
